# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
# API dependencies
import requests
import json
import time

# Pandas dependency
import pandas as pd

# Plotting dependencies
import hvplot.pandas

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sandnessjoen,66.0217,12.6316,288.44,55,100,4.92,NO,1716669397
1,1,sirre,8.3167,39.4833,292.71,62,18,3.08,ET,1716669560
2,2,sur,22.5667,59.5289,304.01,43,51,4.69,OM,1716669361
3,3,grytviken,-54.2811,-36.5092,265.11,74,91,4.35,GS,1716669582
4,4,punta arenas,-53.1500,-70.9167,277.21,87,40,3.60,CL,1716669592


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
cities = city_data_df[["City", "Lat", "Lng"]]

# Display the map
map_plot = cities.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="EsriImagery",
    color = "City"
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
mask = (city_data_df["Max Temp"] > 294) & (city_data_df["Max Temp"] < 299) & (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Humidity"] < 75)

# Drop any rows with null values
ideal_cities_df = city_data_df[mask].dropna()

# Display sample data
print(ideal_cities_df.shape)
print(ideal_cities_df.info())
ideal_cities_df.head()

(11, 10)
<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 86 to 562
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     11 non-null     int64  
 1   City        11 non-null     object 
 2   Lat         11 non-null     float64
 3   Lng         11 non-null     float64
 4   Max Temp    11 non-null     float64
 5   Humidity    11 non-null     int64  
 6   Cloudiness  11 non-null     int64  
 7   Wind Speed  11 non-null     float64
 8   Country     11 non-null     object 
 9   Date        11 non-null     int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 968.0+ bytes
None


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
86,86,pisco,-13.7000,-76.2167,294.18,68,0,4.12,PE,1716670322
102,102,utrera,37.1852,-5.7809,298.98,46,0,3.09,ES,1716670611
168,168,ghormach,35.7306,63.7826,294.22,43,0,3.43,AF,1716671296
207,207,el granada,37.5027,-122.4694,294.29,69,0,7.72,US,1716671704
256,256,yarmouth,41.7057,-70.2286,295.71,52,0,4.12,US,1716672214


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
print(hotel_df.shape)
print(hotel_df.info())
hotel_df.head()

(11, 6)
<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 86 to 562
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        11 non-null     object 
 1   Country     11 non-null     object 
 2   Lat         11 non-null     float64
 3   Lng         11 non-null     float64
 4   Humidity    11 non-null     int64  
 5   Hotel Name  11 non-null     object 
dtypes: float64(2), int64(1), object(3)
memory usage: 616.0+ bytes
None


,City,Country,Lat,Lng,Humidity,Hotel Name
86,pisco,PE,-13.7000,-76.2167,68,
102,utrera,ES,37.1852,-5.7809,46,
168,ghormach,AF,35.7306,63.7826,43,
207,el granada,US,37.5027,-122.4694,69,
256,yarmouth,US,41.7057,-70.2286,52,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

params = {
    "categories": categories,
    "limit": 1,
    "apiKey": geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
        
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pisco - nearest hotel: La Portada
utrera - nearest hotel: Hacienda de Orán
ghormach - nearest hotel: No hotel found
el granada - nearest hotel: Beach House
yarmouth - nearest hotel: The Inn At Cape Cod
tubarjal - nearest hotel: No hotel found
dowlatabad - nearest hotel: هتل آفتاب اصفهان
hampton bays - nearest hotel: Canoe Place
karratha - nearest hotel: Karratha International Hotel
salida - nearest hotel: Hampton Inn & Suites
creston - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
86,pisco,PE,-13.7000,-76.2167,68,La Portada
102,utrera,ES,37.1852,-5.7809,46,Hacienda de Orán
168,ghormach,AF,35.7306,63.7826,43,No hotel found
207,el granada,US,37.5027,-122.4694,69,Beach House
256,yarmouth,US,41.7057,-70.2286,52,The Inn At Cape Cod
334,tubarjal,SA,30.4999,38.2160,41,No hotel found
335,dowlatabad,IR,32.7998,51.6955,27,هتل آفتاب اصفهان
450,hampton bays,US,40.8690,-72.5176,67,Canoe Place
457,karratha,AU,-20.7377,116.8463,29,Karratha International Hotel
470,salida,US,37.7058,-121.0849,49,Hampton Inn & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="OSM",
    color = "Hotel Name",
    size = "Humidity",
    frame_width = 700,
    frame_height = 500
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity)